In [10]:
"""
PDF RAG QA — Jupyter Notebook (single cell version)

Este script implementa um pipeline completo de RAG (Retrieval-Augmented Generation)
para PDFs científicos usando:

- PyMuPDF4LLM para extração e chunking por página
- SentenceTransformers para embeddings semânticos
- Similaridade por cosseno (via dot product normalizado)
- Ollama como LLM local (ex: gemma3:4b)

Projetado para rodar em uma ÚNICA célula de Jupyter Notebook.
"""

# =========================
# Imports
# =========================

import pymupdf4llm
import requests
import numpy as np
from sentence_transformers import SentenceTransformer


# =========================
# Configuração do Sistema
# =========================

# Endpoint local do Ollama
OLLAMA_BASE_URL = "http://localhost:11434/api/generate"

# Modelo LLM local
MODEL_NAME = "gemma3:4b"

# Modelo de embeddings (rápido e robusto para artigos científicos)
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

# Quantidade de chunks recuperados no RAG
TOP_K_MATCHES = 5

# Caminho do PDF (ajuste conforme necessário)
PDF_PATH = "datasets/tattoo.pdf"


# =========================
# Funções Auxiliares
# =========================

def get_embedder():
    """
    Carrega o modelo de embeddings.
    Executar apenas uma vez no notebook.
    """
    return SentenceTransformer(EMBEDDING_MODEL_NAME)


def embed_texts(texts, embedder):
    """
    Gera embeddings normalizados (norma L2 = 1),
    permitindo o uso de produto interno como similaridade cosseno.
    """
    embeddings = embedder.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=True
    )
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)


def chunk_markdown(md_chunks):
    """
    Normaliza a saída do PyMuPDF4LLM para uma lista de strings.
    Cada elemento corresponde a um chunk textual.
    """
    if isinstance(md_chunks, list):
        return [
            chunk if isinstance(chunk, str)
            else chunk.get("text", str(chunk))
            for chunk in md_chunks
        ]
    return [md_chunks]


def retrieve(query, chunks, chunk_embeddings, embedder, k=TOP_K_MATCHES):
    """
    Recupera os k chunks mais similares à query usando similaridade cosseno.
    """
    # Embedding da query
    query_vec = embedder.encode([query], convert_to_numpy=True)
    query_vec = query_vec / np.linalg.norm(query_vec, axis=1, keepdims=True)

    # Similaridade cosseno (dot product)
    similarities = np.dot(chunk_embeddings, query_vec.T).squeeze()

    # Índices dos melhores chunks
    top_indices = np.argsort(similarities)[::-1][:k]

    return [(chunks[i], similarities[i]) for i in top_indices]


def call_ollama(prompt):
    """
    Envia o prompt para o Ollama e retorna a resposta do modelo.
    Streaming desativado para simplicidade no notebook.
    """
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(
        OLLAMA_BASE_URL,
        json=payload,
        timeout=120
    )

    response.raise_for_status()
    return response.json().get("response", "")


# =========================
# Pipeline Principal
# =========================

# 1. Extração e chunking do PDF
print("📄 Extraindo PDF com PyMuPDF4LLM...")
md_chunks = pymupdf4llm.to_markdown(PDF_PATH, page_chunks=True)
chunks = chunk_markdown(md_chunks)
print(f"✅ {len(chunks)} chunks extraídos")

# 2. Geração de embeddings (feito apenas uma vez)
print("\n🧠 Carregando modelo de embeddings...")
embedder = get_embedder()

print("🧠 Gerando embeddings dos chunks...")
chunk_embeddings = embed_texts(chunks, embedder)

# 3. Defina a pergunta (edite esta variável e reexecute a célula)
query = "What are the authors of this work?"

# 4. Recuperação semântica
print("\n🔎 Recuperando contexto relevante...")
top_chunks = retrieve(query, chunks, chunk_embeddings, embedder)

# 5. Construção do contexto para o LLM
context = "\n\n".join(chunk for chunk, _ in top_chunks)

prompt = f"""
Answer the following question using ONLY the context below.
If the answer is not in the context, say so.

Context:
{context}

Question: {query}
Answer:
"""

# 6. Chamada ao LLM
print("\n⏳ Consultando o modelo LLM local (Ollama)...\n")
answer = call_ollama(prompt)

# 7. Exibição da resposta
print("🧠 Resposta:\n")
print(answer)

# 8. Inspeção dos chunks recuperados (debug / avaliação)
print("\n📚 Chunks recuperados:\n")

for i, (chunk, score) in enumerate(top_chunks, 1):
    print(f"--- Chunk {i} | Similaridade: {score:.3f} ---")
    print(chunk[:800])
    print()


📄 Extraindo PDF com PyMuPDF4LLM...
✅ 19 chunks extraídos

🧠 Carregando modelo de embeddings...
🧠 Gerando embeddings dos chunks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔎 Recuperando contexto relevante...

⏳ Consultando o modelo LLM local (Ollama)...

🧠 Resposta:

The authors of this work are not explicitly named in the provided text. It only refers to "the authors" throughout the document.

📚 Chunks recuperados:

--- Chunk 1 | Similaridade: 0.273 ---
This article has been accepted for publication in IEEE Access. This is the author's version which has not been fully edited and

content may change prior to final publication. Citation information: DOI 10.1109/ACCESS.2024.3438557



**KKC Tattoos**





**UUC Tattoos**






















|Animals|Insects|Plants|Architectural|Uncategorized|
|---|---|---|---|---|
|**Crocodile**|**Bee**|**Glass**|**Bridge**|**Iceberg**|
|**Bear**|**Beetle**|**Beetle**|**Castle**|**Joker**|
|**Cow/Bull**|**Ladybug**|**Ladybug**|**Ladybug**|**Robot**|
|**Chicken**|**Chicken**|**Chicken**|**Chicken**|**Ship**|
|**Elephant**|**Elephant**|**Elephant**|**Elephant**|**Ying-Yang**|
|**Hippopotamus**|**Hippopotamus**|**Hippopota

In [7]:
for c in chunks:
    print(c)

This article has been accepted for publication in IEEE Access. This is the author's version which has not been fully edited and

content may change prior to final publication. Citation information: DOI 10.1109/ACCESS.2024.3438557


Date of publication xxxx 00, 0000, date of current version xxxx 00, 0000.


_Digital Object Identifier 10.1109/ACCESS.2017.DOI_

# **Open-Set Tattoo Semantic Segmentation**


1Federal University of Technology - Paraná, Av. Sete de Setembro, 3165, Curitiba, 80230-901, Paraná, Brazil.


Corresponding author: Anderson Brilhador (e-mail: andersonbrilhador@gmail.com).


**ABSTRACT** Tattoos can serve as an essential source of biometric information for public security, aiding
in identifying suspects and victims. In order to automate tattoo classification, tasks like classification
require more detailed image content analysis, such as semantic segmentation. However, a dataset with
appropriate semantic segmentation annotations is currently lacking. Also, there are c